In [1]:
GPU = True
if GPU:
    import cupy as np
else:
    import numpy as np
import import_ipynb
from tqdm import tqdm
from cnn import *
from mnistDataLoader import MnistDataloader

importing Jupyter notebook from cnn.ipynb


In [2]:
mnist_dataloader = MnistDataloader('train-images-idx3-ubyte/train-images-idx3-ubyte', 'train-labels-idx1-ubyte/train-labels-idx1-ubyte', 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

x_train = np.stack(np.array(x_train)).reshape((60000, 1, 28, 28))
y_train = np.stack(np.array(y_train))
x_test = np.stack(np.array(x_test)).reshape((10000, 1, 28, 28))
y_test = np.stack(np.array(y_test))

x_train = (lambda x:x/256)(x_train)
x_test = (lambda x:x/256)(x_test)

def to_one_hot(x, length=10):
    a = [0] * length
    a[int(x)] = 1
    return a
y_train = np.array(list(map(to_one_hot, y_train))).reshape(60000, 10, 1)
y_test = np.array(list(map(to_one_hot, y_test))).reshape(10000, 10, 1)

In [3]:
cnn = [
    Conv2D((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 10),
    Sigmoid()
]


epochs = 10
lr = 0.01
for e in range(epochs):
    cost = 0
    for x, y in tqdm(list(zip(x_train, y_train))):
        output = x
        for l in cnn:
            output = l.forward(output)

        cost += binary_cross_entropy(y, output)

        grad = binary_cross_entropy_gradient(y, output)
        for layer in reversed(cnn):
            grad = layer.backward(grad, lr)

    cost /= len(x_train)
    print(f'Finished epoch {e+1} with cost {cost}')

100%|██████████| 60000/60000 [01:14<00:00, 800.75it/s]

Finished epoch 1 with cost 0.24594642607562817


In [4]:
def predict(x):
    output = x
    for l in cnn:
        output = l.forward(output)
    return np.argmax(output)

def evaluate(x, y):
    res = np.array(list(map(predict, x)))
    ans = np.array(list(map(np.argmax, y)))
    return np.sum(res == ans) / len(res)

In [5]:
idx = np.random.randint(0, len(x_test))

print(predict(x_test[idx]))
print(np.argmax(y_test[idx]))

1
1


In [6]:
print(evaluate(x_test, y_test))

0.653
